In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [3]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(20, 15, bias = True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(15, 10, bias= True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(10, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [4]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [5]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=10000, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 10000)

In [6]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=20, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=20, out_features=15, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.6, inplace=False)
    (8): Linear(in_features=15, out_features=10, bias=True)
    (9): ReLU()
  )
  (classify): Sequential(
    (0): Linear(in_features=10, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [7]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [8]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 53.739706099033356
loss - 33.910582304000854
loss - 33.89159798622131
loss - 33.88821077346802
loss - 33.891886830329895
loss - 33.89867848157883
loss - 33.892412066459656
loss - 33.88836580514908
loss - 33.886214196681976
loss - 33.88957595825195
loss - 33.889761090278625
loss - 33.88747102022171
loss - 33.88984704017639
loss - 33.887325406074524
loss - 33.893911600112915
loss - 33.89974993467331
loss - 33.895221173763275
loss - 33.89426249265671
loss - 33.89461886882782
loss - 33.89052605628967
loss - 33.88882625102997
loss - 33.88729631900787
loss - 33.885713160037994
loss - 33.88565158843994
loss - 33.89199733734131
loss - 33.88773363828659
loss - 33.89388185739517
loss - 33.89512640237808
loss - 33.8867883682251
loss - 33.88788837194443
loss - 33.8856138586998
loss - 33.88507390022278
loss - 33.89667421579361
loss - 33.88410395383835
loss - 33.89231276512146
loss - 33.88796925544739
loss - 33.89167767763138
loss - 33.89127969741821
loss - 33.89119392633438
loss - 33.8939867

In [10]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6156712407294616
